In [2]:
from os.path import exists
import pandas as pd

df_path = "./train.csv.zip"
df = pd.read_csv(df_path)
df= df.sample(n=19000, random_state=0)#randomly pick 19k out of the data
df["store"] = df["store"].astype(str)#set these columns to strings
df["item"] = df["item"].astype(str)

df["date"]= pd.to_datetime(df["date"])

df.sort_values("date",inplace=True)
df.reset_index(inplace=True, drop = True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [3]:
train = df[df["date"]< "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [4]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=['sales'])
test_target = test["sales"]

In [5]:
from upgini import FeaturesEnricher,SearchKey
from upgini.metadata import CVType


enricher = FeaturesEnricher(search_keys={"date":SearchKey.DATE}, cv=CVType.time_series)

enricher.fit(train_features,train_target,eval_set=[(test_features,test_target)])

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=0f4a6962-cfc8-48a2-8af1-3d651ea200d0
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
-


38 relevant feature(s) found with the search keys: ['date']


,,item,0.542757,100.000000,categorical,
,,store,0.190758,100.000000,categorical,
Upgini,Public data,f_weather_date_weather_umap_48_b39cd0c4,0.045142,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_0_1ff14cbf,0.037054,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_24_2e14c9a6,0.032766,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_33_89bb7578,0.022424,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_sin1_847b5db1,0.020552,100.000000,numerical,Free
Upgini,Public data,f_events_date_year_cos1_9014a856,0.015001,100.000000,numerical,Free
Upgini,Public data,f_economic_date_cpi_pca_5_789d07f3,0.005940,100.000000,numerical,Free
Upgini,Public data,f_financial_date_dow_jones_fe02128f,0.004217,100.000000,numerical,Free
Upgini,Public data,f_financial_date_silver_14e835ea,0.003406,100.000000,numerical,Free


Calculating accuracy uplift after enrichment...
|

Quality metrics


,Rows,Baseline mean_squared_error,Enriched mean_squared_error,Uplift
,,,,
Train,15213,331.590507,205.182835,126.407672
Eval 1,3787,581.575525,401.545500,180.030025


In [6]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose= False, allow_writing_files=False, random_state=0)

In [7]:
enriched_train_features = enricher.transform(train_features, keep_input = True)
enriched_test_features = enricher.transform(test_features, keep_input = True)
enriched_train_features.head()

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=985a7d2f-bbb4-4177-9ee9-d206372f2d2d
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
|

Retrieving selected features from data sources...
\
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=8c8f768b-d611-4c9a-8d80-96e69bcf125a
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
|



In [ ]:
model.fit(train_features,train_target)
preds = model.predict(test_features)
eval_metric(test_target.values,preds,"SMAPE")

In [ ]:
model.fit(enriched_train_features,train_target)
better_preds = model.predict(enriched_test_features)
eval_metric(test_target.values,better_preds,"SMAPE")